## Creating and training the CNN model 

### Import the libraries

In [1]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot as plt

# Importing ImageDataGenerator from Keras

In [2]:
# image preprocessing (or) image augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
#import the cnn layers

# Defining the Parameters

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip=True)
#rescale => rescaling pixel value from 0 to 255 to 0 to 1
#shear_range=> counter clock wise rotation(anti clock)

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

# Applying ImageDataGenerator functionality to train dataset

In [6]:
x_train = train_datagen.flow_from_directory(r"D:\FFDDataset\train_set",
                                            target_size=(256,256),
                                            batch_size=32,
                                            class_mode="binary")

Found 436 images belonging to 2 classes.


# Applying ImageDataGenerator functionality to test dataset

In [7]:
x_test = test_datagen.flow_from_directory(r"D:\FFDDataset\test_set",
                                          target_size=(256,256),
                                          batch_size=32,
                                          class_mode="binary")

Found 121 images belonging to 2 classes.


# Importing Model Building Libraries

In [8]:
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam , SGD, RMSprop

In [9]:
x_train.class_indices

{'forest': 0, 'with fire': 1}

# Intializing the model

In [10]:
model = Sequential()

# Adding CNN layers

In [11]:
# add convolution layer

In [12]:
model.add(Convolution2D(32,(3,3),input_shape=(256,256,3),activation="relu"))
# 32 indicates => no of feature detectors
#(3,3)=> kernel size (feature detector size)
#add max pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add flatten layer => input to your ANN  
model.add(Flatten())          

# Add Dense layers

In [13]:
#hidden layer
model.add(Dense(units=300,kernel_initializer="random_uniform",activation="relu"))
model.add(Dense(units=200,kernel_initializer="random_uniform",activation="relu"))
#output layer
model.add(Dense(units=1,kernel_initializer="random_uniform",activation="sigmoid"))

# Configuring the learning process

In [14]:
#compile the model
model.compile(loss=keras.losses.binary_crossentropy,optimizer="adam",metrics=['accuracy'])

# Summarize the model

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 516128)            0         
                                                                 
 dense (Dense)               (None, 300)               154838700 
                                                                 
 dense_1 (Dense)             (None, 200)               60200     
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                        

# Training the model

In [16]:
model.fit(x_train,steps_per_epoch=13,epochs=30,validation_data=x_test,validation_steps=3)
#steps_per_epoch = no of training images/batch size
#validation_steps = no of testing images/batch size

Epoch 1/30
13/13 [==============================] - 48s 4s/step - loss: 1.1523 - accuracy: 0.7426 - val_loss: 0.1376 - val_accuracy: 0.9583
Epoch 2/30
13/13 [==============================] - 24s 2s/step - loss: 0.2809 - accuracy: 0.8861 - val_loss: 0.1445 - val_accuracy: 0.9167
Epoch 3/30
13/13 [==============================] - 22s 2s/step - loss: 0.2506 - accuracy: 0.9109 - val_loss: 0.0680 - val_accuracy: 0.9792
Epoch 4/30
13/13 [==============================] - 24s 2s/step - loss: 0.1443 - accuracy: 0.9381 - val_loss: 0.0736 - val_accuracy: 0.9688
Epoch 5/30
13/13 [==============================] - 22s 2s/step - loss: 0.2340 - accuracy: 0.9035 - val_loss: 0.0631 - val_accuracy: 0.9688
Epoch 6/30
13/13 [==============================] - 22s 2s/step - loss: 0.2004 - accuracy: 0.9109 - val_loss: 0.1092 - val_accuracy: 0.9688
Epoch 7/30
13/13 [==============================] - 23s 2s/step - loss: 0.1592 - accuracy: 0.9356 - val_loss: 0.0325 - val_accuracy: 0.9896
Epoch 8/30
13/13 [==

# Saving the model

In [17]:
model.save("fire.h5")

# Predicitions

In [21]:
#import load model from keras.model
from keras.models import load_model
#import image from keras
from tensorflow.keras.preprocessing import image
import numpy as np
#import cv2
import cv2
#load the saved model
model=load_model("fire.h5")
img=image.load_img(r"C:\Users\Isha\Pictures\Saved Pictures\egfire.jpg")
x=image.img_to_array(img)
res=cv2.resize(x,dsize=(256,256),interpolation=cv2.INTER_CUBIC)
#expand the image shape
x=np.expand_dims(res,axis=0)

In [22]:
pred=model.predict(x)
pred = int(pred[0][0])
print(pred)

1/1 [==============================] - 1s 874ms/step
1


In [ ]:
#loading the saved model
model = load_model("fire.h5")

In [40]:
if pred==1:
    print('Forest fire')
else:
    print('No Fire')

Forest fire


# OpenCV for Video Processing

In [25]:
import cv2
import numpy as np
# importing image function from keras
from keras.preprocessing import image
# importing load_model from keras 
from keras.models import load_model
#importing client from twilio API
from twilio.rest import Client
#importing playsound package from playsound
import playsound

In [42]:
#define video
video = cv2.VideoCapture(0)
#defining the features
name = ['Forest','With fire']

# Sending an alert message through Twilio

In [27]:
if pred==1:
    account_sid = 'AC9c9f37c5348766e642763aa6c4ca3de4'
    auth_token = 'a713206b352a2634b272d9965f6e46c0'
    
    client = Client(account_sid, auth_token)
    message = client.messages \
    .create(body= 'Alert! A Forest fire has been detected.',from_='+18314804693',to='+919498400638')
    
    print(message.sid)
    print('Fire Detected')
    print('Message Sent')

SMc3c7d993b225bced90e219dd7b077a4c
Fire Detected
Message Sent


In [28]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [1]:
#with live video capture
"""
is_frame = True
while is_frame:
    is_frame,frame = video.read()
    if is_frame:
        cv2.imwrite("image.jpg",frame)
        img = image.load_img("image.jpg",target_size=(256,256))
        y = image.img_to_array(img)
        y = np.expand_dims(y,axis=0)
        class_name = ["Forest","With fire"]
        pred_prob = model.predict(y)    
        pred_prob = int(pred_prob[0][0])
        cv2.imshow("Frame", frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        elif pred_prob == 1:
            print(pred_prob)
            print("predicted class is ",str(class_name[1]))
            cv2.putText(frame, "Detected class is " + str(class_name[1]), (10, 100),
					cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 5)
            account_sid = 'AC9c9f37c5348766e642763aa6c4ca3de4'
            auth_token = 'a713206b352a2634b272d9965f6e46c0'
            client = Client(account_sid, auth_token)
            message = client.messages \
            .create(body= 'Alert! A Forest fire has been detected.',from_='+18314804693',to='+919498400638')
            print(message.sid)
            print('Fire Detected')
            print('Message Sent')
        elif pred_prob == 0:
            print(pred_prob)
            print("predicted class is ",str(class_name[0]))
            cv2.putText(frame, "Detected class is " + str(class_name[0]), (10, 100),
					cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 5)
            print("No Danger")
        
    else:
        break
cv2.destroyAllWindows()
video.release()
"""

'\nis_frame = True\nwhile is_frame:\n    is_frame,frame = video.read()\n    if is_frame:\n        cv2.imwrite("image.jpg",frame)\n        img = image.load_img("image.jpg",target_size=(256,256))\n        y = image.img_to_array(img)\n        y = np.expand_dims(y,axis=0)\n        class_name = ["Forest","With fire"]\n        pred_prob = model.predict(y)    \n        pred_prob = int(pred_prob[0][0])\n        cv2.imshow("Frame", frame)\n        if cv2.waitKey(25) & 0xFF == ord(\'q\'):\n            break\n        elif pred_prob == 1:\n            print(pred_prob)\n            print("predicted class is ",str(class_name[1]))\n            cv2.putText(frame, "Detected class is " + str(class_name[1]), (10, 100),\n\t\t\t\t\tcv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 5)\n            account_sid = \'AC9c9f37c5348766e642763aa6c4ca3de4\'\n            auth_token = \'a713206b352a2634b272d9965f6e46c0\'\n            client = Client(account_sid, auth_token)\n            message = client.messages       